# Set up run

**To do:**


*   Change testing cells for compatibility with generator





In [0]:
#-- Set up configurations / parameters
retrain = False # retrain previously existing model
ndown = 4 # number of 'down' steps
ninit = 32 #number of channels to start with
dropout_frac = 0.2 # dropout fraction
n_batch = 10
n_epochs = 60
n_test = 50
ratio = 727 # penalization ratio for GL and non-GL points based on smaller dataaset
mod_lbl = 'atrous' #'unet'
if mod_lbl == 'unet':
  mod_str = '{0}_{1}init_{2}down_drop{3:.1f}'.format(mod_lbl,ninit,ndown,
                                                        dropout_frac)
elif mod_lbl == 'atrous':
  mod_str = '{0}_{1}init_drop{2:.1f}'.format(mod_lbl,ninit,dropout_frac)
else:
  print('model label not matching.')
print(mod_str)

atrous_32init_drop0.2


In [0]:
#-- Import Modules
import os
import imp
import numpy as np
import matplotlib.pyplot as plt 
from google.colab import drive
import keras
from keras import backend as K
from keras.preprocessing import image
from tensorflow.python.client import device_lib
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

Using TensorFlow backend.


In [0]:
#-- Mount Google Drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#-- Directory setup
colabdir = '/content/gdrive/My Drive/Colab Notebooks/'
output_dir = '/content/gdrive/My Drive/GL_Learning/'
ddir = '/content/gdrive/Shared drives/GROUNDING_LINE_TEAM_DRIVE/ML_Yara/geocoded_v1/'
train_dir = os.path.join(ddir,'train_n%i.dir'%n_test)
test_dir = os.path.join(ddir,'test_n%i.dir'%n_test)

In [0]:
#-- Get list of images
fileList = os.listdir(train_dir)
train_list = [f for f in fileList if (f.endswith('.npy') and f.startswith('coco'))]
fileList = os.listdir(test_dir)
test_list = [f for f in fileList if (f.endswith('.npy') and f.startswith('coco'))]

In [0]:
#-- set ID_list to be passed onto data generator class
ID_list = [os.path.join(train_dir,f) for f in train_list]
N = len(ID_list)
print(N)

#-- read 1 file to get dimensions
im = np.load(ID_list[0])
h,wi,ch = im.shape
print(h,wi,ch)

#-- randomly set aside 10% of data for validation
valid_ii = random.sample(range(N),int(0.1*N))
train_ii = np.setdiff1d(np.arange(N),valid_ii)
#-- setup generator
data_generation = imp.load_source('data_generation',os.path.join(colabdir,'data_generation.py'))
from data_generation import DataGenerator
training_generator = DataGenerator(list(np.array(ID_list)[train_ii]),batch_size=n_batch, dim=(h,wi), 
	n_channels=ch, shuffle=True, ratio=ratio)
validation_generator = DataGenerator(list(np.array(ID_list)[valid_ii]),batch_size=n_batch, dim=(h,wi), 
	n_channels=ch, shuffle=True, ratio=ratio)

n train: 4874, n test 20


In [0]:
#-- Upgrade U-Net model from TF1 to TF2
!tf_upgrade_v2 \
  --infile /content/gdrive/My\ Drive/Colab\ Notebooks/unet_model.py \
  --outfile /content/gdrive/My\ Drive/Colab\ Notebooks/unet_model_v2.py

In [0]:
#-- Import model
mod_module = imp.load_source('unet_model',os.path.join(colabdir,'unet_model.py'))
#-- set up model
if mod_lbl == 'unet':
  print('loading unet model')
  model = mod_module.unet_model_double_dropout(height=h,width=wi,channels=channels, 
                                        n_init=ninit,n_layers=ndown,
                                        drop=dropout_frac)
elif mod_lbl == 'atrous':
  print("loading atrous model")
  model = mod_module.unet_model_atrous_double_dropout(height=h,width=wi,
                                                channels=channels,
                                                n_filts=ninit,
                                                drop=dropout_frac)
else:
  print('Model label not correct.')

In [0]:
#-- compile imported model
model.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'],sample_weight_mode="temporal")

In [0]:
#-- checkpoint file
chk_file = os.path.join(output_dir,'{0}_weights.h5'.format(mod_str))

#-- if file exists, read model from file
if os.path.isfile(chk_file):
  print('Check point exists; loading model from file.')
  #-- load weights
  model.load_weights(chk_file)
else:
  print('Model does not previously exist.')

In [0]:
#-- Train the model
if (retrain) or (not os.path.isfile(chk_file)):
  print('Training model...')
  #-- create checkpoint
  model_checkpoint = keras.callbacks.ModelCheckpoint(chk_file, monitor='loss',
                                                     verbose=1, 
                                                     save_best_only=True)
  lr_callback = ReduceLROnPlateau(monitor='acc', factor=0.5, patience=5,
                                  verbose=1, mode='auto', min_delta=0.0001, 
                                  cooldown=0, min_lr=0)
  # es_callback = EarlyStopping(monitor='val_loss',min_delta=0.0001, patience=5,
  #    verbose=1, mode='auto')
  #-- now fit the model
  history = model.fit_generator(generator=training_generator,
                                validation_data=validation_generator,
                                use_multiprocessing=True,
                                workers=4, epochs=n_epochs,
                                shuffle=True, verbose=1,
                                callbacks=[lr_callback,model_checkpoint])
                                #callbacks=[lr_callback,es_callback,model_checkpoint])

  #-- save history to file
  outfile = open(os.path.join(output_dir,
                        '{0}_history.txt'
                        .format(mod_str)),'w')
  outfile.write('Epoch loss\tval_loss\tacc\tval_acc\n')
  for i in range(len(history.history['loss'])):
      outfile.write('%i\t%f\t%f\t%f\t%f\n'%(i,history.history['loss'][i],history.history['val_loss'][i],\
          history.history['acc'][i],history.history['val_acc'][i]))
  outfile.close()

  #-- Make plots for training history
  for item,name in zip(['acc','loss'],['Accuracy','Loss']):
    fig = plt.figure(1,figsize=(8,6))
    plt.plot(history.history[item])
    plt.plot(history.history['val_%s'%item])
    plt.title('Model %s'%name)
    plt.ylabel(name)
    plt.xlabel('Epochs')
    plt.legend(['Training', 'Validation'], loc='upper left')
    plt.savefig(os.path.join(output_dir,
                        '{0}_history.pdf'
                        .format(mod_str)),format='pdf')
    plt.close(fig)

In [0]:
print('Model is trained. Running on Train data...')
#-------------------------------
#-- First run on training data
#-------------------------------
out_imgs = model.predict(train_imgs, batch_size=1, verbose=1)
out_imgs = out_imgs.reshape(out_imgs.shape[0],h,wi,out_imgs.shape[2])
print(out_imgs.shape)

#-- make output directory
out_dir = os.path.join(output_dir,'Train_predictions.dir',
                       '{0}.dir'.format(mod_str))
if (not os.path.isdir(out_dir)):
  os.mkdir(out_dir)

#-- save output images (every 4th image. Not interested in augmented cases)
for i,f in enumerate(train_list):
  im = image.array_to_img(out_imgs[aug_num*i]) 
  im.save(os.path.join(out_dir,f.replace('coco.tif','pred.png')))

In [0]:
print(np.max(out_imgs))
print(np.min(out_imgs))

In [0]:
#-- Read test data
test_imgs = np.ones((n_test,h,wi,nch))
for i,f in enumerate(img_list[n_train:]):
    #-- read image
    raster = rasterio.open(os.path.join(test_dir,f))
    test_imgs[i,:,:,0] = raster.read(1).real
    test_imgs[i,:,:,1] = raster.read(1).imag

In [0]:
#-------------------------------
#-- Run on test data
#-------------------------------
out_imgs = model.predict(test_imgs, batch_size=1, verbose=1)
out_imgs = out_imgs.reshape(out_imgs.shape[0],h,wi,out_imgs.shape[2])
print(out_imgs.shape)

#-- make output directory
out_dir = os.path.join(output_dir,'Test_predictions.dir',
                       '{0}.dir'.format(mod_str))
if (not os.path.isdir(out_dir)):
  os.mkdir(out_dir)

#-- save output images
for i,f in enumerate(test_list):
  im = image.array_to_img(out_imgs[i]) 
  im.save(os.path.join(out_dir,f.replace('coco.tif','pred.png')))